In [1]:
import feedparser

from bs4 import BeautifulSoup

from whoosh.index import create_in
from whoosh.fields import *
from whoosh.qparser import QueryParser, FuzzyTermPlugin
from whoosh import highlight

import calendar
from datetime import datetime

In [2]:
import feedparser

NewsFeed = feedparser.parse("https://thegeneralist.substack.com/feed")

entry = NewsFeed.entries[0]
print(entry)

{'title': 'Affirm: The Morality of Money', 'title_detail': {'type': 'text/plain', 'language': None, 'base': 'https://thegeneralist.substack.com/feed', 'value': 'Affirm: The Morality of Money'}, 'summary': 'Affirm in 1 minute PayPal co-founder Max Levchin is back. Affirm, the &#8220;buy now, pay later&#8221; solution led by Levchin is slated to list on the Nasdaq before the year is out. The wisdom of the company&#8217;s mooted $10 billion valuation may depend on the perspective of the investor in question. Is Affirm a lender, selling a commoditized product? Or is it a payments network, capitalizing on changing consumer preferences to steal share from big banks? For Levchin, Affirm&#8217;s disruption of traditional finance is not just a commercial opportunity but a moral imperative.', 'summary_detail': {'type': 'text/html', 'language': None, 'base': 'https://thegeneralist.substack.com/feed', 'value': 'Affirm in 1 minute PayPal co-founder Max Levchin is back. Affirm, the &#8220;buy now, p

In [3]:
print(entry['title'])
print(entry['author'])
print(entry['summary'])
print(entry['link'])
print(entry['published_parsed'])
content = entry['content'][0]['value']
print(content)

Affirm: The Morality of Money
MDA Gabriele
Affirm in 1 minute PayPal co-founder Max Levchin is back. Affirm, the &#8220;buy now, pay later&#8221; solution led by Levchin is slated to list on the Nasdaq before the year is out. The wisdom of the company&#8217;s mooted $10 billion valuation may depend on the perspective of the investor in question. Is Affirm a lender, selling a commoditized product? Or is it a payments network, capitalizing on changing consumer preferences to steal share from big banks? For Levchin, Affirm&#8217;s disruption of traditional finance is not just a commercial opportunity but a moral imperative.
https://thegeneralist.substack.com/p/affirm-the-morality-of-money
time.struct_time(tm_year=2020, tm_mon=12, tm_mday=9, tm_hour=16, tm_min=21, tm_sec=33, tm_wday=2, tm_yday=344, tm_isdst=0)
<div class="captioned-image-container"><figure><a class="image-link image2 image2-1132-1456" href="https://cdn.substack.com/image/fetch/f_auto,q_auto:good,fl_progressive:steep/https%

In [4]:
# Turn HTML into clean text for nice searching

from bs4 import BeautifulSoup
tree = BeautifulSoup(content)

# This validates and cleans HTML but Substack HTML is already fine
# pretty = tree.prettify()
# print(pretty)

pure_text = tree.get_text("\n")
print(pure_text)

Affirm in 1 minute
PayPal co-founder Max Levchin is back. Affirm, the “buy now, pay later” solution led by Levchin is slated to list on the Nasdaq before the year is out. The wisdom of the company’s mooted $10 billion valuation may depend on the perspective of the investor in question. Is Affirm a lender, selling a commoditized product? Or is it a payments network, capitalizing on changing consumer preferences to steal share from big banks? For Levchin, Affirm’s disruption of traditional finance is not just a commercial opportunity but a moral imperative. 
Affirm’s revenue for the financial year ending June 2020 stood at $509.5 million, a 93% improvement from the year prior. Such growth requires contextualization, though — 28% came from a single merchant partner: Peloton. Like many other public debutants this year, Affirm has yet to post a profit, with net losses of $120.5 million in 2019 and $112.6 million in 2020. 
Founders Fund, Khosla Ventures, Lightspeed Venture Partners, and Jasm

In [5]:
# Next, build a Whoosh database and search engine
# https://whoosh.readthedocs.io/en/latest/quickstart.html#a-quick-introduction

# Define schema (pretty simple)
# Add documents from all top Substacks
# Build a searcher
# Show results
# Highlight bits from the thinkpieces that match
# Optional: add stemming for better searching
# Later, add support for other newsletters with an RSS feed
# Maybe, later, offer `more_like_this` so people can rabbit-hole in

# Use case: I want to find all articles from my favorite writers that include the term
# "BNPL" since I want to research "Buy Now, Pay Later" in fintech.
# Or I want to find all thinkpieces that mentioned Google in the last year

In [6]:
# Create a list of all RSSes to search

# First, Substacks. Getting their RSS feeds is pretty simple/systematic.
substack_domains = [
    "thegeneralist",
    "danco",
    "diff",
    "nbt",
    "platformer",
    "notboring",
    "sariazout",
    "digitalnative",
    "jamesonstartups",
    "breakingsmart",
    "artofgig",
    "theskip",
    "gwern"
]

# Feeds are, e.g., https://thegeneralist.substack.com/feed
substack_feeds = ["https://{0}.substack.com/feed".format(domain) for domain in substack_domains]

# Now add some custom RSS feeds
# Medium feeds are medium.com/feed/@user or medium.com/feed/publication
custom_feeds = [
    "https://stratechery.com/feed/",
    "https://www.profgalloway.com/feed",
    "https://eugene-wei.squarespace.com/blog?format=rss",
    "https://medium.com/feed/@superwuster",
    "https://commoncog.com/blog/rss",
    "https://www.lennyrachitsky.com/feed",
    "https://medium.com/feed/bloated-mvp",
    "https://daringfireball.net/feeds/main",
    "https://wongmjane.com/api/feed/rss",
    "https://fourweekmba.com/feed",
]

# Unite all feeds into one
all_feeds = substack_feeds + custom_feeds
all_feeds

# We'll read in the feeds from each of these. Get RSS feed at 
# https://thegeneralist.substack.com/feed

# NewsFeed = feedparser.parse("https://thegeneralist.substack.com/feed")

['https://thegeneralist.substack.com/feed',
 'https://danco.substack.com/feed',
 'https://diff.substack.com/feed',
 'https://nbt.substack.com/feed',
 'https://platformer.substack.com/feed',
 'https://notboring.substack.com/feed',
 'https://sariazout.substack.com/feed',
 'https://digitalnative.substack.com/feed',
 'https://jamesonstartups.substack.com/feed',
 'https://breakingsmart.substack.com/feed',
 'https://artofgig.substack.com/feed',
 'https://theskip.substack.com/feed',
 'https://gwern.substack.com/feed',
 'https://stratechery.com/feed/',
 'https://www.profgalloway.com/feed',
 'https://eugene-wei.squarespace.com/blog?format=rss',
 'https://medium.com/feed/@superwuster',
 'https://commoncog.com/blog/rss',
 'https://www.lennyrachitsky.com/feed',
 'https://medium.com/feed/bloated-mvp',
 'https://daringfireball.net/feeds/main',
 'https://wongmjane.com/api/feed/rss',
 'https://fourweekmba.com/feed']

In [7]:
# Convenience function to safely get an item from a dict.
# If the key doesn't exist, just returns none
def safe_get(obj, key):
    if obj.has_key(key):
        return obj[key]
    else:
        return None

In [8]:
# Build the database
schema = Schema(
    title=TEXT(stored=True),
    author=TEXT(stored=True),
    publication=TEXT(stored=True),
    summary=TEXT(stored=True),
    url=TEXT(stored=True),
    published=DATETIME(stored=True),
    content=TEXT(stored=True))
index = create_in("whoosh_index2", schema)
writer = index.writer()

# Read every item from our RSS feeds into there
# Call writer.add_document() repeatedly for each item 

for feed_url in all_feeds:
    print(feed_url)
    news_feed = feedparser.parse(feed_url)
    
    # NOTE: we can only get the last few entries from this RSS feed.
    # Substack doesn't seem to show anything older than the last 20.
    # So we should build in a system to start caching these.
    
#     print(len(news_feed.entries))

    for entry in news_feed.entries:
        
        # Get publication name. This is in the feed's `feed` field, along with other metadata
        publication = None
        metadata = safe_get(news_feed, 'feed')
        if metadata is not None:
            publication = safe_get(metadata, 'title')

        # Clean up the date into a normal datetime
        clean_datetime = datetime.fromtimestamp(calendar.timegm(entry['published_parsed']))
        
        # Most feeds put the main content in `content`,
        # but a rare few like Eugene Wei put it in `summary`
        # (in which case `content` is empty). With this logic, let's get a single `content` field.
        body_text = None
        # See if `content` exists
        content_holder = safe_get(entry, 'content')
        if content_holder is not None:
            # We have content; fill it in
            content_tree = BeautifulSoup(content_holder[0]['value'])
            body_text = content_tree.get_text(" ", strip=True)
        else:
            # No content provided. `summary` must hold all the text.
            summary_tree = BeautifulSoup(safe_get(entry, 'summary'))
            body_text = summary_tree.get_text(" ", strip=True)

        writer.add_document(
            title=safe_get(entry, 'title'),
            author=safe_get(entry, 'author'),
            publication=publication,
            summary=safe_get(entry, 'summary'),
            url=safe_get(entry, 'link'),
            published=clean_datetime,
            content=body_text)

print("DONE!")
writer.commit()

https://thegeneralist.substack.com/feed
https://danco.substack.com/feed
https://diff.substack.com/feed
https://nbt.substack.com/feed
https://platformer.substack.com/feed
https://notboring.substack.com/feed
https://sariazout.substack.com/feed
https://digitalnative.substack.com/feed
https://jamesonstartups.substack.com/feed
https://breakingsmart.substack.com/feed
https://artofgig.substack.com/feed
https://theskip.substack.com/feed
https://gwern.substack.com/feed
https://stratechery.com/feed/
https://www.profgalloway.com/feed
https://eugene-wei.squarespace.com/blog?format=rss
https://medium.com/feed/@superwuster
https://commoncog.com/blog/rss
https://www.lennyrachitsky.com/feed
https://medium.com/feed/bloated-mvp
https://daringfireball.net/feeds/main
https://wongmjane.com/api/feed/rss
https://fourweekmba.com/feed
DONE!


In [9]:
# For convenience, we're overriding the standard fragment formatter
class BracketFormatter(highlight.Formatter):
    """Puts square brackets around the matched terms.
    """

    def format_token(self, text, token, replace=False):
        # Use the get_text function to get the text corresponding to the
        # token
        tokentext = highlight.get_text(text, token, replace)

        # Return the text as you want it to appear in the highlighted
        # string
        return "[[%s]]" % tokentext

In [27]:
# Try searching
from whoosh.qparser import QueryParser, MultifieldParser
import whoosh.qparser as qparser

search_term = "Twitter"

with index.searcher() as searcher:
    parser = QueryParser("content", index.schema)
    # Allow fuzzy matching (EDIT: kinda screws things up)
    # parser.add_plugin(FuzzyTermPlugin())
    # Allow searching for entire phrases w/ single quotes, like 'microsoft teams'
    parser.add_plugin(qparser.SingleQuotePlugin())
    
    query = parser.parse(search_term)
    results = searcher.search(query, limit=None)
    
    # Highlighting settings
    # This provides more context characters around the searched-for text
    results.fragmenter.surround = 50
    results.fragmenter.maxchars = 500
    
    # Surround matched tags with brackets
    results.formatter = BracketFormatter()
    
    # Convert each Hit into a dict
    def extract_hit_info(hit):
        return {
            'title': hit.get('title'),
            'publication': hit.get('publication'),
            'author': hit.get('author'),
            'url': hit.get('url'),
            'highlights': hit.highlights("content", top=3)
        }
    
    hit_list = [extract_hit_info(h) for h in results]
    
    print(hit_list)

[{'title': 'How Twitter and the Internet Broke the News', 'publication': 'Digital Native', 'author': 'Rex Woodbury', 'url': 'https://digitalnative.substack.com/p/how-twitter-and-the-internet-broke', 'highlights': 'creates more cultural and societal value than [[Twitter]].) The story is even worse on a per-user basis. [[Twitter]]’s ARPU (Average Revenue Per User) for North America is...There’s some debate about whether [[Twitter]] is a social network. [[Twitter]] does have a social graph—you follow people, and they follow...enormous value for the media ecosystem—off of [[Twitter]]’s back—and they’re reaping all of the rewards. [[Twitter]] reaps nothing. [[Twitter]] needs to earn a share of revenue from the traffic it drives'}, {'title': 'Twitter & Elections', 'publication': 'Prof Galloway', 'author': 'Scott Galloway', 'url': 'https://www.profgalloway.com/twitter-elections/', 'highlights': 'that [[Twitter]] had work to do on direct response advertising (they have...and revenues increased

In [11]:
# Experimental

news_feed = feedparser.parse("https://diff.substack.com/feed")
print(news_feed['feed'])

{'title': 'The Diff', 'title_detail': {'type': 'text/plain', 'language': None, 'base': 'https://diff.substack.com/feed', 'value': 'The Diff'}, 'subtitle': 'Inflections in finance and tech', 'subtitle_detail': {'type': 'text/html', 'language': None, 'base': 'https://diff.substack.com/feed', 'value': 'Inflections in finance and tech'}, 'links': [{'rel': 'alternate', 'type': 'text/html', 'href': 'https://diff.substack.com'}, {'href': 'https://diff.substack.com/feed', 'rel': 'self', 'type': 'application/rss+xml'}], 'link': 'https://diff.substack.com', 'image': {'href': 'https://cdn.substack.com/image/fetch/w_256,c_limit,f_auto,q_auto:good,fl_progressive:steep/https%3A%2F%2Fbucketeer-e05bbc84-baa3-437e-9518-adb32be77984.s3.amazonaws.com%2Fpublic%2Fimages%2F5b730ae4-aa35-496e-8198-965a187e2e43_600x600.png', 'title': 'The Diff', 'title_detail': {'type': 'text/plain', 'language': None, 'base': 'https://diff.substack.com/feed', 'value': 'The Diff'}, 'links': [{'rel': 'alternate', 'type': 'text/

In [12]:
# TODO: read from the thinkpiecer module and use it here, instead of writing custom code

In [20]:
z = {'a':5, 'b':3}

In [25]:
print(z.get('c'))

None
